In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

## 협업플랫폼

### Data Preprocessing

In [4]:
target_year = 2021
target_month = 7

pref = "D:/Github/Work/Tableau/Collaboration Tool Usage/"
hr = pd.read_excel(f"D:/Github/Work/Tableau/직원 명단_210731.xlsx")

filenames = glob.glob(f"{pref}Raw Data_Collaboration Tool Usage/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
data = pd.concat(dfs, ignore_index=True)

In [6]:
hr = hr.drop_duplicates()
hr = hr.rename({"사번":"num", "성명":"name", "EMAIL":"email", "본부명(조직단위기준)":"hq", "원소속명":"dep", "직급":"rank", "구분":"type"}, axis=1)
hr = hr[["num", "name", "email", "hq", "dep", "rank", "type"]]

# 휴직 상태인 직원은 제외합니다.
hr = hr[~hr["dep"].str.contains("휴직")]
# 현장의 "hq"는 "현장"으로 변환합니다.
hr.loc[hr["type"].str.contains("현장"), "hq"] = "현장"
# 신규준비는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(신규준비)", "", regex=False)
# 대기는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(대기)", "", regex=False)
# "(연구실)", "(착공준비)"는 해당 팀 소속으로 간주합니다.
hr["dep"] = hr["dep"].str.replace("(연구실)", "", regex=False)
hr["dep"] = hr["dep"].str.replace("(착공준비)", "", regex=False)
# 지사 소속 직원은 제외합니다.
hr = hr[hr["type"]!="지사"]
hr.loc[hr["dep"].str.contains("건축PAM"), "dep"] = "건축PAM"
hr.loc[hr["dep"].str.contains("인프라PAM"), "dep"] = "인프라PAM"

hr.loc[hr["num"]==1101290, "hq"] = "개발영업본부"

email2name = {email:name for email, name in zip(hr["email"], hr["name"])}
email2dep = {email:dep for email, dep in zip(hr["email"], hr["dep"])}
email2hq = {email:hq for email, hq in zip(hr["email"], hr["hq"])}

In [7]:
# data = data.drop(["comment_id"], axis=1)
# data = data.fillna(0)
# data[["post_subject_size", "comment_body_size"]] = data[["post_subject_size", "comment_body_size"]].astype("int")
data["post_created_at"] = data["post_created_at"].apply(lambda x:pd.to_datetime(x))
data["comment_created_at"] = data["comment_created_at"].apply(lambda x:pd.to_datetime(x))
# data["post_created_at"] = pd.to_datetime(data["post_created_at"])
# data["post_created_at"] = pd.to_datetime(data["post_created_at"])

In [10]:
data[(data["post_id"] == 3042887262052256600) & (data["project_name"].str.contains("-보고"))]

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
293351,2981038588610011468,미래혁신본부-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikl...",2021-06-30,NaN,NaN,NaN,NaT
299629,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.043367e+18,339.0,hikli@hdc-dvp.com,2021-07-01
299630,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.043389e+18,585.0,mej@hdc-dvp.com,2021-07-01
299631,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.043390e+18,150.0,hikli@hdc-dvp.com,2021-07-01
299632,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.048660e+18,899.0,5969@hdc-dvp.com,2021-07-08
299633,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.052222e+18,683.0,6363@hdc-dvp.com,2021-07-13
299634,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.057250e+18,1658.0,kyungkeun.yoo@hdc-dvp.com,2021-07-20


In [12]:
data[(data["post_id"] == 3042887262052256600) & (data["project_name"].str.contains("-보고"))]["post_cc"].tolist()

["['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']"]

### 최근 부서별 사용량.csv

In [60]:
data[(data["post_from"] == "6363@hdc-dvp.com") & (data["post_created_at"].dt.year == target_year) & (data["post_created_at"].dt.month == target_month)].drop_duplicates(["post_id"])

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
279405,2590735541878145948,IT-신청,3038300713542430032,36,483,6363@hdc-dvp.com,['4100422@hdc-dvp.com'],"['wkcho@hdc-dvp.com','mej@hdc-dvp.com','han96m...",2021-06-24,3.038311e+18,19.0,wkcho@hdc-dvp.com,2021-06-24
280773,2597671184417881648,Go-혁신,3023891813996122407,50,614,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hd...",2021-06-04,3.026161e+18,224.0,6363@hdc-dvp.com,2021-06-07
280800,2597671184417881648,Go-혁신,3032593156325597879,47,1081,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['6299@hdc-dvp.com','kyungkeun.yoo@hdc-dvp.com...",2021-06-16,NaN,NaN,NaN,NaT
293351,2981038588610011468,미래혁신본부-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikl...",2021-06-30,NaN,NaN,NaN,NaT
293411,2981892333338475758,고객관리혁신-HelpDesk,3027404296285317447,73,795,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['6048@hdc-dvp.com','hyejin.kim@hdc-dvp.com','...",2021-06-09,3.028366e+18,648.0,4100422@hdc-dvp.com,2021-06-10


In [ ]:
# 최근 1개월 간 작성된 업무만 뽑습니다.
recent_posts = data[(data["post_created_at"].dt.year == target_year) & (data["post_created_at"].dt.month == target_month)]
# 타사 직원이 작성한 업무는 제거합니다.
# recent_posts = recent_posts[recent_posts["post_from"] != "0"]
# 중복된 업무는 제거합니다.
recent_posts = recent_posts.drop_duplicates(["post_id"])
recent_posts = pd.merge(recent_posts, hr, left_on="post_from", right_on="email", how="outer")
recent_posts = recent_posts[["project_name", "hq", "dep", "name", "post_body_size"]]
recent_post = recent_posts.groupby(["hq", "dep", "name"]).count()["post_body_size"]
recent_post_size = recent_posts.groupby(["hq", "dep", "name"]).sum()["post_body_size"]

# 최근 1개월 간 작성된 댓글만 뽑습니다.
recent_comments = data[(data["comment_created_at"].dt.year==target_year) & (data["comment_created_at"].dt.month==target_month)]
# 타사 직원이 작성한 댓글은 제거합니다.
# recent_comments = recent_comments[recent_comments["commenter"] != "0"]
recent_comments = pd.merge(recent_comments, hr, left_on="commenter", right_on="email", how="outer")
recent_comments = recent_comments[["project_name", "hq", "dep", "name", "comment_body_size"]]
recent_comment = recent_comments.groupby(["hq", "dep", "name"]).count()["comment_body_size"]
recent_comment_size = recent_comments.groupby(["hq", "dep", "name"]).sum()["comment_body_size"]

temp = pd.merge(recent_post, recent_comment, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, recent_post_size, left_index=True, right_index=True)
merge = pd.merge(temp, recent_comment_size, left_index=True, right_index=True)
merge.columns = ["post", "comment", "post_size", "comment_size"]

hr_temp = hr[~hr["hq"].str.contains("회장직속|부회장직속")]
idx = pd.MultiIndex.from_tuples(list(hr_temp.groupby(["hq", "dep", "name"]).groups))
recent = pd.DataFrame(data=0, index=idx, columns=["post", "comment", "post_size", "comment_size"])

for _, row in merge.iterrows():
    recent.loc[row.name] = row
    
recent = recent.reset_index(drop=False).rename({"level_0":"hq", "level_1":"dep", "level_2":"name"}, axis=1)
recent = recent[~recent["dep"].str.contains("회장|사장|본부|상임자문|비상근")]

In [6]:
# recent.to_csv(f"{pref}최근 부서별 사용량.csv", encoding="euc-kr", index=False)
recent.head()

,hq,dep,name,post,comment,post_size,comment_size
0,개발영업본부,H1프로젝트,김 현석,0.0,0.0,0.0,0.0
1,개발영업본부,H1프로젝트,김 호중,0.0,7.0,0.0,126.0
2,개발영업본부,H1프로젝트,박 주원,1.0,4.0,975.0,713.0
3,개발영업본부,H1프로젝트,윤 기락,0.0,0.0,0.0,0.0
4,개발영업본부,H1프로젝트,임 도훈,0.0,1.0,0.0,53.0


### 보고방별 보고 건수.csv

In [10]:
prj2head = {"회장-보고":hr.loc[hr["dep"]=="회장", "email"].item(), "부회장-보고":hr.loc[hr["dep"]=="부회장", "email"].item(), "대표이사사장-보고":hr.loc[hr["dep"]=="대표이사사장", "email"].item(), "개발영업본부-보고":hr.loc[hr["dep"]=="개발영업본부", "email"].item(), "건설본부-보고":hr.loc[hr["dep"]=="건설본부", "email"].item(), "경영본부-보고":hr.loc[hr["dep"]=="경영본부", "email"].item(), "미래혁신본부-보고":hr.loc[hr["dep"]=="미래혁신본부", "email"].item()}
head2prj = {value:key for key, value in prj2head.items()}

recent_cmts = data[(data["comment_created_at"].dt.year == target_year) & (data["comment_created_at"].dt.month == target_month)]
recent_cmts = recent_cmts[(recent_cmts["project_name"].isin(prj2head))]
recent_cmts = recent_cmts[(recent_cmts["commenter"].isin(head2prj))]
recent_cmts = recent_cmts.drop_duplicates(["post_id", "commenter"], keep="first")

# escal = pd.merge(recent_cmts, hr[["email", "hq", "dep", "name"]], left_on="post_from", right_on="email")
# escal = escal[["project_name", "post_id", "hq", "dep", "name"]]

In [13]:
recent_cmts.groupby(["post_id", ])

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
279503,2591135457748217935,부회장-보고,3023289752373719490,154,2511,dgpark@hdc-dvp.com,['dgpark@hdc-dvp.com'],"['beauiful@hdc-dvp.com','5953@hdc-dvp.com','dc...",2021-06-03,3.023931e+18,42.0,dckim@hdc-dvp.com,2021-06-04
279505,2591135457748217935,부회장-보고,3026816377448447631,36,816,kmhanjs@hdc-dvp.com,['kmhanjs@hdc-dvp.com'],"['beauiful@hdc-dvp.com','dckim@hdc-dvp.com','j...",2021-06-08,3.026843e+18,176.0,dckim@hdc-dvp.com,2021-06-08
279508,2591135457748217935,부회장-보고,3026931025075322896,35,2024,youngjoo@hdc-dvp.com,['youngjoo@hdc-dvp.com'],"['beauiful@hdc-dvp.com','dckim@hdc-dvp.com','k...",2021-06-08,3.027400e+18,190.0,dckim@hdc-dvp.com,2021-06-09
279510,2591135918625451217,개발영업본부-보고,3007138549003951592,55,2383,kyh@hdc-dvp.com,"['kyh@hdc-dvp.com','jh4793@hdc-dvp.com','hadh@...","['suna.yang@hdc-dvp.com','park1@hdc-dvp.com','...",2021-05-12,3.031104e+18,61.0,park1@hdc-dvp.com,2021-06-14
279513,2591135918625451217,개발영업본부-보고,3021115021076156344,111,365,prhdc@hdc-dvp.com,['prhdc@hdc-dvp.com'],"['park1@hdc-dvp.com','suna.yang@hdc-dvp.com']",2021-05-31,3.021569e+18,110.0,park1@hdc-dvp.com,2021-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293322,2981038588610011468,미래혁신본부-보고,3027616282322983239,40,344,cho6502@hdc-dvp.com,['cho6502@hdc-dvp.com'],"['oh5555@hdc-dvp.com','ally.noh@hdc-dvp.com','...",2021-06-09,3.027619e+18,19.0,hikli@hdc-dvp.com,2021-06-09
293329,2981038588610011468,미래혁신본부-보고,3031924865497368956,70,1549,yjlee@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','yjlee@hdc-dvp.com']","['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikl...",2021-06-15,3.031928e+18,21.0,hikli@hdc-dvp.com,2021-06-15
293334,2981038588610011468,미래혁신본부-보고,3032496049340359293,47,1046,powerohs@hdc-dvp.com,['powerohs@hdc-dvp.com'],"['oh5555@hdc-dvp.com','jylee@hdc-dvp.com','mej...",2021-06-16,3.032596e+18,15.0,hikli@hdc-dvp.com,2021-06-16
293335,2981038588610011468,미래혁신본부-보고,3033364174816914630,111,244,wh@hdc-dvp.com,"['wh@hdc-dvp.com','mej@hdc-dvp.com']","['4001341@hdc-dvp.com','hikli@hdc-dvp.com']",2021-06-17,3.033367e+18,47.0,hikli@hdc-dvp.com,2021-06-17


In [11]:
escal[escal["name"]=="조 도연"]

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at,email,hq,dep,name
32,2591136577788543151,대표이사사장-보고,3022462670303557928,53,584,cho6502@hdc-dvp.com,['cho6502@hdc-dvp.com'],"['kshjh@hdc-dvp.com','oh5555@hdc-dvp.com','all...",2021-06-02,3.023258e+18,19.0,hikli@hdc-dvp.com,2021-06-03,cho6502@hdc-dvp.com,미래혁신본부,HR혁신팀,조 도연
33,2591136577788543151,대표이사사장-보고,3022462670303557928,53,584,cho6502@hdc-dvp.com,['cho6502@hdc-dvp.com'],"['kshjh@hdc-dvp.com','oh5555@hdc-dvp.com','all...",2021-06-02,3.026801e+18,105.0,kshjh@hdc-dvp.com,2021-06-08,cho6502@hdc-dvp.com,미래혁신본부,HR혁신팀,조 도연
34,2981038588610011468,미래혁신본부-보고,3027616282322983239,40,344,cho6502@hdc-dvp.com,['cho6502@hdc-dvp.com'],"['oh5555@hdc-dvp.com','ally.noh@hdc-dvp.com','...",2021-06-09,3.027619e+18,19.0,hikli@hdc-dvp.com,2021-06-09,cho6502@hdc-dvp.com,미래혁신본부,HR혁신팀,조 도연


In [9]:
data[data["post_id"] == 3036842012076339346]

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
279705,2591136577788543151,대표이사사장-보고,3036842012076339346,68,227,cho6502@hdc-dvp.com,['cho6502@hdc-dvp.com'],"['oh5555@hdc-dvp.com','kshjh@hdc-dvp.com']",2021-06-22,3.036852e+18,410.0,oh5555@hdc-dvp.com,2021-06-22


In [73]:
prj2head = {"회장-보고":hr.loc[hr["dep"]=="회장", "email"].item(), "부회장-보고":hr.loc[hr["dep"]=="부회장", "email"].item(), "대표이사사장-보고":hr.loc[hr["dep"]=="대표이사사장", "email"].item(), "개발영업본부-보고":hr.loc[hr["dep"]=="개발영업본부", "email"].item(), "건설본부-보고":hr.loc[hr["dep"]=="건설본부", "email"].item(), "경영본부-보고":hr.loc[hr["dep"]=="경영본부", "email"].item(), "미래혁신본부-보고":hr.loc[hr["dep"]=="미래혁신본부", "email"].item()}
head2prj = {value:key for key, value in prj2head.items()}

# 최근 1개월 간 작성된 댓글만 뽑습니다.
recent_cmts = data[(data["comment_created_at"].dt.year == target_year) & (data["comment_created_at"].dt.month == target_month)]
# 위 프로젝트에서 작성된 업무 및 댓글만 뽑습니다.
escal = recent_cmts[recent_cmts["project_name"].isin(prj2head.keys())]
# 업무별로 댓글을 남긴 사람을 모두 리스트로 만듭니다.
escal["commenters"] = escal.apply(lambda x:escal[escal["post_id"] == x["post_id"]]["commenter"].tolist(), axis=1)
# 보고 아이템 기준으로 중복된 보고를 제거합니다.
escal = escal.drop_duplicates(["post_id"])
# 각 프로젝트마다의 보고 받는 이를  "head" column으로 설정합니다.
escal["head"] = escal["project_name"].map(prj2head)
# 댓글을 남긴 사람들의 리스트에 보고 받는 이를 포함시킵니다.
escal.apply(lambda x:x["commenters"].append(x["head"]), axis=1)
# 댓글을 남긴 사람들 중 본부장 이상만 남깁니다.
escal["commenters"] = escal["commenters"].apply(lambda x:list(set(x) & set(prj2head.values())))
# 본부장 이상에 대해 각각 담당하는 보고방에 대응시킵니다.
escal["heads"] = escal["commenters"].apply(lambda x:[head2prj[el] for el in x])
escal = pd.merge(escal, hr[["email", "hq", "dep", "name"]], left_on="post_from", right_on="email")
escal = escal[["hq", "dep", "name", "heads", "post_id"]]

C:\Users\00006363\Anaconda3\envs\tf2.3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [75]:
escal[escal["name"] == "조 도연"]

,hq,dep,name,heads,post_id
40,미래혁신본부,HR혁신팀,조 도연,"[대표이사사장-보고, 미래혁신본부-보고]",3022462670303557928
41,미래혁신본부,HR혁신팀,조 도연,[대표이사사장-보고],3036842012076339346
42,미래혁신본부,HR혁신팀,조 도연,[미래혁신본부-보고],3027616282322983239


In [72]:
df = pd.DataFrame()
for _, row in escal.iterrows():
    for el in row["heads"]:
        df = df.append({"hq":row["hq"], "dep":row["dep"], "name":row["name"], "project_name":el}, ignore_index=True)
escalation = df.groupby(["project_name", "hq", "dep", "name"], as_index=False).size()

# 보고 수가 0인 프로젝트를 데이터에 추가합니다.
missing_set = set(prj2head.keys()) - set(escalation["project_name"])
for missing in missing_set:
    escalation = escalation.append({"project_name":missing, "hq":"-", "dep":"-", "name":"-", "size":0}, ignore_index=True)

KeyError: 'post_id'

In [59]:
# escalation.to_csv(f"{pref}보고방별 보고 건수.csv", encoding="euc-kr")
escalation.head()

,project_name,hq,dep,name,size
0,개발영업본부-보고,개발영업본부,도시정비1팀,이 용민,1
1,개발영업본부-보고,개발영업본부,도시정비기획사업소,권 영현,1
2,개발영업본부-보고,개발영업본부,민간수주1팀,한 상현,3
3,개발영업본부-보고,개발영업본부,복합개발팀,김 지환,2
4,개발영업본부-보고,개발영업본부,복합개발팀,김 진택,1


### 연월별 사용량.csv

In [9]:
data_posts = data.drop_duplicates(["post_id"])
post_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).size()
post_month.name = "post"
post_size_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["post_body_size"]
post_size_month.name = "post_size"

comment_month = data.groupby(pd.Grouper(key="comment_created_at", freq="M")).size()
comment_month.name = "comment"
comment_size_month = data.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["comment_body_size"]
comment_size_month.name = "comment_size"

temp = pd.merge(post_month, comment_month, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, post_size_month, left_index=True, right_index=True, how="inner")
month = pd.merge(temp, comment_size_month, left_index=True, right_index=True, how="inner")

In [10]:
month.to_csv(f"{pref}연월별 사용량.csv", encoding="euc-kr")
month.tail()

,post,comment,post_size,comment_size
2021-02-28,8031.0,10315,58007365.0,1567083
2021-03-31,9949.0,9938,78496434.0,1499882
2021-04-30,9590.0,9212,74380327.0,1948546
2021-05-31,8954.0,9945,65439333.0,2121287
2021-06-30,10499.0,12357,73883408.0,1585326


### 연월별 보고 건수.csv

In [11]:
bogo = data[data["project_name"].isin(prj2head.keys())]
bogo = bogo.drop_duplicates(["post_id"])
bogo_month = bogo.groupby(pd.Grouper(key="post_created_at", freq="M")).size()

In [12]:
bogo_month.to_csv(f"{pref}연월별 보고 건수.csv", encoding="euc-kr")
bogo_month.tail()

post_created_at
2021-02-28     24
2021-03-31     35
2021-04-30     35
2021-05-31    128
2021-06-30     82
Freq: M, dtype: int64

# H-PIC

In [12]:
# filenames = glob.glob("./FINAL/전체/*.csv")
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename))
# hpic = pd.concat(dfs, ignore_index=True)

# hpic["post_id"] = hpic["post_id"].astype("str")
# hpic["post_id"] = hpic["post_id"].str[-10:]

# hpic = hpic.drop(["project_id", "post_subject_size", "comment_id"], axis=1)

# hpic["post_created_at"] = hpic["post_created_at"].apply(lambda x:pd.to_datetime(x))
# hpic["comment_created_at"] = hpic["comment_created_at"].apply(lambda x:pd.to_datetime(x))
# # 가장 바깥의 따옴표를 제거합니다.
# hpic["post_to"] = hpic["post_to"].map(eval)
# hpic["post_cc"] = hpic["post_cc"].map(eval)
# hpic["to"] = hpic["post_to"] + hpic["post_cc"]

# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic = hpic.drop(["post_to", "post_cc"], axis=1)

# hpic.apply(lambda x:x["to"].remove(x["post_from"]) if x["post_from"] in x["to"] else x["to"], axis=1)
# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic["to"] = hpic["to"].apply(lambda x:x[0])

# hpic = hpic.rename({"post_from":"from"}, axis=1)

# hpic["from_hq"] = hpic["from"].map(email2hq)
# hpic["from_dep"] = hpic["from"].map(email2dep)
# hpic["from_name"] = hpic["from"].map(email2name)

# hpic["to_hq"] = hpic["to"].map(email2hq)
# hpic["to_dep"] = hpic["to"].map(email2dep)
# hpic["to_name"] = hpic["to"].map(email2name)

# hpic["commenter"] = hpic["commenter"].map(email2name)

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# for post in set(hpic["post_id"]):
#     hpic[hpic["post_id"]==post]

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# hpic = hpic[["from_hq", "from_dep", "from_name", "from_comment", "to_hq", "to_dep", "to_name", "to_comment", "comment_created_at"]]

In [13]:
# hpic.to_csv("H-PIC 멘토-멘티 댓글 작성 날짜 및 용량_-210228.csv", encoding="euc-kr")
# hpic.head()

In [89]:
filenames = glob.glob(f"{pref}Raw Data_Collaboration Tool Usage/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
df = pd.concat(dfs, ignore_index=True)
# df = pd.read_csv("D:/Github/Work/Tableau/Collaboration Tool Usage/TEST/협업플랫폼_전체_210601-210630.csv")

In [90]:
df = df.rename({"project_id":"prj_id", "project_name":"prj", "comment_id":"cmt_id", "comment_body_size":"cmt_body_size", "commenter":"cmter", "comment_created_at":"cmt_created_at"}, axis=1)
df[["prj_id", "post_id", "cmt_id"]] = df[["prj_id", "post_id", "cmt_id"]].astype("str")
df["post_created_at"] = pd.to_datetime(df["post_created_at"])
df["cmt_created_at"] = pd.to_datetime(df["cmt_created_at"])
# df["cmt_created_at"] = df["cmt_created_at"].apply(lambda x:pd.to_datetime(x))

df = df.sort_values(by=["post_created_at", "cmt_created_at"])

prjs = df.iloc[:, :2]
prjs = prjs.drop_duplicates(["prj_id"], keep="last")

posts = df[df["cmt_created_at"].notna()]
posts = pd.concat([posts.iloc[:, 1], posts.iloc[:, 2:9], posts.iloc[:, 12]], axis=1)
# posts = posts.drop_duplicates(["post_id", "prj"], keep="last")

cmts = pd.concat([df.iloc[:, 2], df.iloc[:, 9:]], axis=1)
cmts = cmts[cmts.notna().sum(axis=1) == 5]

# prjs.to_excel("D:/Github/Work/Tableau/Collaboration Tool Usage/TEST/prjs.xlsx", index=False)
# posts.to_excel("D:/Github/Work/Tableau/Collaboration Tool Usage/TEST/posts.xlsx", index=False)
# cmts.to_excel("D:/Github/Work/Tableau/Collaboration Tool Usage/TEST/cmts.xlsx", index=False)

In [91]:
posts[(posts["post_id"] == "3042887262052256600")]

,prj,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,cmt_created_at
299629,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-01
299630,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-01
299631,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-01
299632,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-08
299633,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-13
299634,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,2021-07-20


In [65]:
posts[(posts["post_id"] == "3042887262052256600") & (posts["prj_id"] == "2591136577788543151")]

KeyError: 'prj_id'